# Initial parameters

In [11]:
import nglview

pdbCode = "1aki"

# Download Structure

In [12]:
# Import module
from biobb_io.api.pdb import Pdb

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode+'.pdb'
prop = {
    'pdb_code': pdbCode
}

#Create and launch bb
Pdb(output_pdb_path=downloaded_pdb,
    properties=prop).launch()

2021-01-23 17:50:07,958 [MainThread  ] [INFO ]  Downloading: 1aki from: https://files.rcsb.org/download/1aki.pdb
2021-01-23 17:50:09,054 [MainThread  ] [INFO ]  Writting pdb to: /Users/hospital/biobb_tutorials/biobb_amber/biobb_amber/notebooks/biobb_amber_notebook/1aki.pdb
2021-01-23 17:50:09,059 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']


# Visualizing 3D structure

In [13]:
# Show protein
view = nglview.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

# Fix structure

### Using pdb4amber, a PDB analyzer and fixer for Amber simulations

In [14]:
# Check & Fix Protein Structure
# Import module
from biobb_model.model.fix_side_chain import FixSideChain

# Create prop dict and inputs/outputs
fixed_pdb = pdbCode+'_fixed.pdb'

# Create and launch bb
FixSideChain(input_pdb_path=downloaded_pdb,
             output_pdb_path=fixed_pdb).launch()

2021-01-23 17:50:10,563 [MainThread  ] [INFO ]  check_structure -i 1aki.pdb -o 1aki_fixed.pdb --force_save fixside --fix ALL

2021-01-23 17:50:10,566 [MainThread  ] [INFO ]  Exit code 0

2021-01-23 17:50:10,568 [MainThread  ] [INFO ]  ===============================================================================
=                   MDWeb structure checking utility v1.1                     =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-20                  =

Structure 1aki.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution: 0.0 A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  129
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1001

Running fixside. Options: --fix ALL
No residues with missing or unknown side chain atoms found
Structure not modified, saving due to --force_save option
Final Num. models: 1
Final Num. chains: 1 (A: Protein)
Final Num. residues:  12

0

In [15]:
# Create prop dict and inputs/outputs
prop = {
    "remove_tmp": True
}
output_pdb4amber_path = 'structure.pdb4amber.pdb'

# Import module
from biobb_amber.pdb4amber.pdb4amber import Pdb4amber

# Create and launch bb
Pdb4amber(input_pdb_path=fixed_pdb,
            output_pdb_path=output_pdb4amber_path,
            properties=prop).launch()

2021-01-23 17:50:10,703 [MainThread  ] [INFO ]  Creating 2e2b2c79-fd9b-4b94-bc64-a670481d87c5 temporary folder
2021-01-23 17:50:10,704 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-01-23 17:50:11,679 [MainThread  ] [INFO ]  pdb4amber  -i 1aki_fixed.pdb -o structure.pdb4amber.pdb

2021-01-23 17:50:11,684 [MainThread  ] [INFO ]  Exit code 0

2021-01-23 17:50:11,686 [MainThread  ] [INFO ]  
Summary of pdb4amber for: 1aki_fixed.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames


---------- Mising heavy atom(s)

None

2021-01-23 17:50:11,688 [MainThread  ] [INFO ]  Removed: 2e2b2c79-fd9b-4b94-bc64-a670481d87c5


0

# Generate Topology
#### IMPORTANT: the previous pdb4amber building block is adding the needed CONNECT lines to the PDB so that this step can automatically identify and add the disulfide bonds to the system topology. 

In [16]:
# Create prop dict and inputs/outputs
prop = {
    "forcefield" : ["protein.ff14SB"],
    "remove_tmp": False
}
output_pdb_path = 'structure.leap.pdb'
output_top_path = 'structure.leap.top'
output_crd_path = 'structure.leap.crd'

# Import module
from biobb_amber.leap.leap_gen_top import LeapGenTop

# Create and launch bb
LeapGenTop(input_pdb_path=output_pdb4amber_path,
           output_pdb_path=output_pdb_path,
           output_top_path=output_top_path,
           output_crd_path=output_crd_path,
           properties=prop).launch()

2021-01-23 17:50:11,751 [MainThread  ] [INFO ]  Creating 217d57ce-29d2-4795-9503-0f8a9725ec8d temporary folder
2021-01-23 17:50:11,754 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-01-23 17:50:12,086 [MainThread  ] [INFO ]  tleap  -f 217d57ce-29d2-4795-9503-0f8a9725ec8d/leap.in

2021-01-23 17:50:12,088 [MainThread  ] [INFO ]  Exit code 0

2021-01-23 17:50:12,088 [MainThread  ] [INFO ]  -I: Adding /anaconda3/envs/biobb_amber/dat/leap/prep to search path.
-I: Adding /anaconda3/envs/biobb_amber/dat/leap/lib to search path.
-I: Adding /anaconda3/envs/biobb_amber/dat/leap/parm to search path.
-I: Adding /anaconda3/envs/biobb_amber/dat/leap/cmd to search path.
-f: Source 217d57ce-29d2-4795-9503-0f8a9725ec8d/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./217d57ce-29d2-4795-9503-0f8a9725ec8d/leap.in
----- Source: /anaconda3/envs/biobb_amber/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /anaconda3/envs/biobb_amber/dat/leap/cm

0

In [17]:
import nglview
import ipywidgets

# Show protein
view = nglview.show_structure_file(output_pdb_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [18]:
# Import module
from biobb_amber.leap.leap_solvate import LeapSolvate

# Create prop dict and inputs/outputs
prop = {
    "forcefield" : ["protein.ff14SB"],
    #"water_type" : "SPCBOX",
    "water_type": "TIP3PBOX",
    "distance_to_molecule": "9.0",
    #"neutralise" : False,
    #"positive_ions_number": 5,
    #"negative_ions_number": 5,    
    "box_type": "truncated_octahedron",
    "remove_tmp": False
}
output_solv_pdb_path = 'structure.solv.pdb'
output_solv_top_path = 'structure.solv.parmtop'
output_solv_crd_path = 'structure.solv.crd'

# Create and launch bb
LeapSolvate(input_pdb_path=output_pdb_path,
           output_pdb_path=output_solv_pdb_path,
           output_top_path=output_solv_top_path,
           output_crd_path=output_solv_crd_path,
           properties=prop).launch()

2021-01-23 17:50:12,261 [MainThread  ] [INFO ]  Creating 6abd7570-c7eb-49f3-8019-8a3096d798ff temporary folder
2021-01-23 17:50:12,263 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-01-23 17:50:13,481 [MainThread  ] [INFO ]  tleap  -f 6abd7570-c7eb-49f3-8019-8a3096d798ff/leap.in

2021-01-23 17:50:13,483 [MainThread  ] [INFO ]  Exit code 0

2021-01-23 17:50:13,487 [MainThread  ] [INFO ]  -I: Adding /anaconda3/envs/biobb_amber/dat/leap/prep to search path.
-I: Adding /anaconda3/envs/biobb_amber/dat/leap/lib to search path.
-I: Adding /anaconda3/envs/biobb_amber/dat/leap/parm to search path.
-I: Adding /anaconda3/envs/biobb_amber/dat/leap/cmd to search path.
-f: Source 6abd7570-c7eb-49f3-8019-8a3096d798ff/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./6abd7570-c7eb-49f3-8019-8a3096d798ff/leap.in
----- Source: /anaconda3/envs/biobb_amber/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /anaconda3/envs/biobb_amber/dat/leap/cm

0

In [19]:
# Import module
from biobb_amber.leap.leap_add_ions import leap_add_ions

# Create prop dict and inputs/outputs
prop = {
    "forcefield" : ["protein.ff14SB"],
    "neutralise" : False,
    "positive_ions_number": 5,
    "negative_ions_number": 5,    
    "box_type": "truncated_octahedron",
    "remove_tmp": False
}
output_ions_pdb_path = 'structure.ions.pdb'
output_ions_top_path = 'structure.ions.parmtop'
output_ions_crd_path = 'structure.ions.crd'

# Create and launch bb
leap_add_ions(input_pdb_path=output_solv_pdb_path,
           output_pdb_path=output_ions_pdb_path,
           output_top_path=output_ions_top_path,
           output_crd_path=output_ions_crd_path,
           properties=prop)

2021-01-23 17:50:13,613 [MainThread  ] [INFO ]  Creating b8f021d8-7303-44e0-aaa6-89f17e3a5787 temporary folder
2021-01-23 17:50:13,620 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-01-23 17:50:16,807 [MainThread  ] [INFO ]  tleap  -f b8f021d8-7303-44e0-aaa6-89f17e3a5787/leap.in

2021-01-23 17:50:16,810 [MainThread  ] [INFO ]  Exit code 0

2021-01-23 17:50:16,811 [MainThread  ] [INFO ]  -I: Adding /anaconda3/envs/biobb_amber/dat/leap/prep to search path.
-I: Adding /anaconda3/envs/biobb_amber/dat/leap/lib to search path.
-I: Adding /anaconda3/envs/biobb_amber/dat/leap/parm to search path.
-I: Adding /anaconda3/envs/biobb_amber/dat/leap/cmd to search path.
-f: Source b8f021d8-7303-44e0-aaa6-89f17e3a5787/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./b8f021d8-7303-44e0-aaa6-89f17e3a5787/leap.in
----- Source: /anaconda3/envs/biobb_amber/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /anaconda3/envs/biobb_amber/dat/leap/cm

0

In [20]:
# Show protein
view = nglview.show_structure_file(output_solv_pdb_path)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein')
view.add_representation(repr_type='ball+stick', selection='solvent')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [21]:
# Create prop dict and inputs/outputs
prop = {
    "simulation_type" : "minimization",
    "mdin" : { 
        'maxcyc' : 500
    },
    "remove_tmp": False
}
output_min_traj_path = 'sander.min.x'
output_min_rst_path = 'sander.min.rst'
output_min_log_path = 'sander.min.log'

# Import module
from biobb_amber.sander.sander_mdrun import SanderMDRun

# Create and launch bb
SanderMDRun(input_top_path=output_ions_top_path,
            input_crd_path=output_ions_crd_path,
            output_traj_path=output_min_traj_path,
            output_rst_path=output_min_rst_path,
            output_log_path=output_min_log_path,
            properties=prop).launch()

2021-01-23 17:50:17,533 [MainThread  ] [INFO ]  Creating 2d8289f6-bc0c-4384-8c49-9d4499b4b86d temporary folder
2021-01-23 17:50:17,545 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-01-23 17:52:22,326 [MainThread  ] [INFO ]  sander -O -i 2d8289f6-bc0c-4384-8c49-9d4499b4b86d/sander.mdin -p structure.ions.parmtop -c structure.ions.crd -r sander.min.rst -o sander.min.log -x sander.min.x

2021-01-23 17:52:22,328 [MainThread  ] [INFO ]  Exit code 0



0

# Checking Energy Minimization results
Checking energy minimization results. Plotting potential energy by time during the minimization process.

In [22]:
# New bb with process_minout.perl (included in ambertools). Take summary.ENERGY



# Heating system

In [26]:
# Create prop dict and inputs/outputs
prop = {
    "simulation_type" : "heat",
    "mdin" : { 
        'nstlim' : 500
    },
    "remove_tmp": False
}
output_heat_traj_path = 'sander.heat.x'
output_heat_rst_path = 'sander.heat.rst'
output_heat_log_path = 'sander.heat.log'

# Import module
from biobb_amber.sander.sander_mdrun import SanderMDRun

# Create and launch bb
SanderMDRun(input_top_path=output_ions_top_path,
            input_crd_path=output_min_rst_path,
            output_traj_path=output_heat_traj_path,
            output_rst_path=output_heat_rst_path,
            output_log_path=output_heat_log_path,
            properties=prop).launch()

2021-01-23 17:53:15,797 [MainThread  ] [INFO ]  Creating a6c01f13-0a1f-482b-a107-0189ab00590b temporary folder
2021-01-23 17:53:15,800 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments


KeyboardInterrupt: 

# Checking Heating results
Checking heating results. 

In [24]:
# New bb with process_mdout.perl (included in ambertools). Take summary.ENERGY


In [25]:
# Create prop dict and inputs/outputs
prop = {
    "simulation_type" : 'npt',
    "mdin" : { 
        'nstlim' : 500
    },
    "remove_tmp": False
}
output_npt_traj_path = 'sander.npt.x'
output_npt_rst_path = 'sander.npt.rst'
output_npt_log_path = 'sander.npt.log'

# Import module
from biobb_amber.sander.sander_mdrun import SanderMDRun

# Create and launch bb
SanderMDRun(input_top_path=output_solv_top_path,
            input_crd_path=output_heat_rst_path,
            output_traj_path=output_npt_traj_path,
            output_rst_path=output_npt_rst_path,
            output_log_path=output_npt_log_path,
            properties=prop).launch()

2021-01-23 17:52:22,766 [MainThread  ] [INFO ]  Creating 0338ee38-1af9-4822-a5ea-8137bc5b75ba temporary folder
2021-01-23 17:52:22,768 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-01-23 17:52:22,956 [MainThread  ] [INFO ]  sander -O -i 0338ee38-1af9-4822-a5ea-8137bc5b75ba/sander.mdin -p structure.solv.parmtop -c sander.heat.rst -r sander.npt.rst -o sander.npt.log -x sander.npt.x

2021-01-23 17:52:22,957 [MainThread  ] [INFO ]  Exit code 1



1